In [1]:
import mechanicalsoup
from dotenv import load_dotenv
import os
import pymongo
from datetime import datetime
from zoneinfo import ZoneInfo
import sys


load_dotenv('.env')


browser = mechanicalsoup.StatefulBrowser()

def login(username, password):
    browser.open("https://members.foodcoop.com/services/login/")

    browser.select_form('form[action="/services/login/"]')
    browser["username"] = username
    browser["password"] = password
    resp = browser.submit_selected()

    if resp.url == "https://members.foodcoop.com/services/login/":
        raise Exception("Login failed")
    else:
        print("Login successful")

In [2]:
def logout():
    browser.open("https://members.foodcoop.com/services/logout/")

In [3]:
logout()
login(os.getenv("COOP_USERNAME"), os.getenv("COOP_PASSWORD"))

Login successful


In [4]:
def current_time_hourly():
    now = datetime.now(ZoneInfo("America/New_York"))
    return now.replace(minute=0, second=0, microsecond=0)

In [5]:
shifts = []
browser.open("https://members.foodcoop.com/services/shifts/")
#browser.open("https://members.foodcoop.com/services/shifts/0/0/0/2024-09-07/")

<Response [200]>

In [6]:
while True:
    page = browser.get_current_page()
    
    shift_elements = page.find_all(class_='shift')
    
    for shift in shift_elements:
        time = shift.b.text
        shift.b.extract()
        type = shift.text.replace('🥕', '').strip()
        date = shift.parent.p.b.text.replace('\xa0', ' ')
        
        shifts.append({
            "date": date,
            "time": time,
            "type": type,
            "created": current_time_hourly()
        })
    
    print(f"Processed {len(shift_elements)} shifts. Total shifts: {len(shifts)}")
    
    try:
        browser.follow_link(text="Next Week →")
    except mechanicalsoup.LinkNotFoundError:
        print("No more weeks to process.")
        break

print(f"Total shifts collected: {len(shifts)}")

Processed 5 shifts. Total shifts: 5
Processed 11 shifts. Total shifts: 16
Processed 173 shifts. Total shifts: 189
Processed 378 shifts. Total shifts: 567
Processed 516 shifts. Total shifts: 1083
Processed 673 shifts. Total shifts: 1756
No more weeks to process.
Total shifts collected: 1756


In [7]:
try:
    client = pymongo.MongoClient(os.getenv("MONGODB_URI"))
    client.admin.command('ping')
    print("Successfully connected to MongoDB")
except pymongo.errors.ConnectionFailure as e:
    print(f"Failed to connect to MongoDB: {e}", file=sys.stderr)
    sys.exit(1)

Successfully connected to MongoDB


In [8]:
shift_collection = client['coop']['shifts']

try:
    result = shift_collection.insert_many(shifts)
    print(f"Successfully inserted {len(result.inserted_ids)} documents")
except pymongo.errors.BulkWriteError as e:
    print(f"Error inserting documents: {e}", file=sys.stderr)
    sys.exit(1)

Successfully inserted 1756 documents
